In [ ]:
import pandas as pd
import re
from google.colab import files
!pip install konlpy
from konlpy.tag import Okt

# 파일 경로 설정
file_path = r'/content/drive/MyDrive/combined_reviews2 - 11월 15일.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 불용어 리스트 (필요에 따라 수정)
stop_words = ['한','에서','하다','를','한다는','이라는','까지','에서는','으로','인데','이고','에요','이를','있다','여서','이다',
              '부터','지인','짜다','라고','되었다','고함','고해']

def preprocess_tweet(tweet):
    # Check if the tweet is a string
    if isinstance(tweet, str):
        # 링크 제거
        tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
        # 줄바꿈 문자 및 특수 문자, 숫자 제거
        tweet = re.sub(r'[\n\t]+|[^가-힣\s]', '', tweet)
        # 토큰화
        tokens = okt.morphs(tweet, stem=True)  # 동사 원형으로 변환
        # 불용어 제거 및 길이가 2 이상인 토큰만 필터링
        filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
        return filtered_tokens
    else:
        # Handle non-string values (e.g., NaN) by returning an empty list
        return []

# 트윗 내용 전처리 적용
df['전처리된 트윗'] = df['트윗 내용'].apply(preprocess_tweet)

# 리포스트 수 변환 함수
def convert_reposts(repost_str):
    if pd.isna(repost_str):
        return 0
    if isinstance(repost_str, (int, float)):
        return int(repost_str)
    repost_str = str(repost_str)
    if '만' in repost_str:
        return int(float(repost_str.replace('만', '')) * 10000)
    elif '천' in repost_str:
        return int(float(repost_str.replace('천', '')) * 1000)
    try:
        return int(float(repost_str))
    except ValueError:
        return 0

# 답글 제한 여부 변환 함수
def convert_reply_limit(reply_limit):
    if pd.isna(reply_limit):
        return 0
    return 0 if str(reply_limit) == '제한 없음' else 1

# 변환 적용
df['리포스트 수'] = df['리포스트 수'].apply(convert_reposts)
df['답글 제한 여부'] = df['답글 제한 여부'].apply(convert_reply_limit)

#새로운 열 '라벨링' 추가 및 값 1로 설정 << 여기 변경해주세요!! 불법이면 1, 일반이면 0
df['라벨링'] = 0
# 리스트를 문자열로 변환
df['전처리된 트윗_문자열'] = df['전처리된 트윗'].apply(lambda x: ' '.join(x))

output_path = '/content/drive/MyDrive/processed_reviews.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"처리된 데이터가 {output_path}에 저장되었습니다.")

df

처리된 데이터가 /content/drive/MyDrive/processed_reviews.csv에 저장되었습니다.


,계정 이름,트윗 내용,리포스트 수,답글 제한 여부,작성한 포스트 수,전처리된 트윗,라벨링,전처리된 트윗_문자열
0,다인의 편의점이것저것_채다인,"GS25에서 사본 하이디라오 토마토 & 야채 간편훠궈(8,900원)발열팩 넣어 찬물...",5200,0,6.5만,"[사본, 하이디, 라오, 토마토, 야채, 간편, 훠궈, 발열, 넣다, 찬물, 부어,...",0,사본 하이디 라오 토마토 야채 간편 훠궈 발열 넣다 찬물 부어 기다리다 바로 먹다 ...
1,다인의 편의점이것저것_채다인,https://x.com/worKer_sw/status/185687193731174...,10,0,6.5만,[],0,
2,다인의 편의점이것저것_채다인,이거 맛있어서 자매품 곱창 볶음밥도 예약주문함,65,0,6.5만,"[맛있다, 자매, 곱창, 볶음밥]",0,맛있다 자매 곱창 볶음밥
3,다인의 편의점이것저것_채다인,"CU 감자탕볶음 후식주먹밥(1,600원)이거 들깨가루의 꼬소한 맛이 완전히 킥.ㅋㅋ...",95,0,6.5만,"[감자탕, 볶음, 후식, 주먹밥, 들깨, 가루, 꼬소, 완전하다, 참기름, 매콤, ...",0,감자탕 볶음 후식 주먹밥 들깨 가루 꼬소 완전하다 참기름 매콤 조르다 국물 들깨 가...
4,다인의 편의점이것저것_채다인,"세븐일레븐 매일 데르뜨 밤티라미수(4,900원)ㅋㅋ흑백요리사 밤티라미수를 노골적으로...",4900,0,6.5만,"[세븐일레븐, 매일, 데르뜨, 티라미수, 흑백, 요리사, 티라미수, 노골, 따르다,...",0,세븐일레븐 매일 데르뜨 티라미수 흑백 요리사 티라미수 노골 따르다 미투 상품 맞다 ...
...,...,...,...,...,...,...,...,...
972,kube,이 스너프킨 후드 담요 탐라의 누군가는 좋아하겠지.,3500,0,4.3만,"[스너프, 후드, 담요, 탐라, 누군가, 좋아하다]",0,스너프 후드 담요 탐라 누군가 좋아하다
973,kube,안나수이 새 향수 디자인이랑 이름 좀 봐. 물론 썬데겠지만 자꾸 순대로 읽혀. 향조...,5100,0,4.3만,"[안나, 수이, 향수, 디자인, 이랑, 이름, 보다, 물론, 써다, 자꾸, 순대, ...",0,안나 수이 향수 디자인 이랑 이름 보다 물론 써다 자꾸 순대 읽히다 달달 자체 스트...
974,kube,이 그림 포근한 색채도 좋지만 조용한 반려동물이라는 제목도 마음에 들어.\n Joh...,2300,0,4.3만,"[그림, 포근하다, 색채, 좋다, 조용하다, 반려동물, 제목, 마음, 들다]",0,그림 포근하다 색채 좋다 조용하다 반려동물 제목 마음 들다
975,kube,"일본 아마존 다이슨 스토어 블프 세일 가격 38,280엔.. 38,280엔... 한...",7700,0,4.3만,"[일본, 아마존, 다이슨, 스토어, 블프, 가격, 한국, 반값, 이네, 역시, 한국...",0,일본 아마존 다이슨 스토어 블프 가격 한국 반값 이네 역시 한국 블프 가짜


In [ ]:
import pandas as pd
import re
from konlpy.tag import Okt
from nltk.corpus import stopwords

# 파일 경로 설정
file_path = r'/content/drive/MyDrive/명확한 데이터 - 시트1.csv'

# CSV 파일 읽기
df1 = pd.read_csv(file_path)

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 불용어 리스트 (필요에 따라 수정)
stop_words = stoplist=['한','에서','하다','를','한다는','이라는','까지','에서는','으로','인데','이고','에요','이를','있다','여서','이다',
                       '부터','지인','짜다','라고','되었다','고함','고해']
def preprocess_tweet(tweet):
    # 링크 제거
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    # 줄바꿈 문자 및 특수 문자, 숫자 제거
    tweet = re.sub(r'[\n\t]+|[^가-힣\s]', '', tweet)
    # 토큰화
    tokens = okt.morphs(tweet, stem=True)  # 동사 원형으로 변환
    # 불용어 제거 및 길이가 2 이상인 토큰만 필터링
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
    return filtered_tokens

# 트윗 내용 전처리 적용
df1['전처리된 트윗'] = df1['트윗 내용'].apply(preprocess_tweet)

# 리포스트 수 변환 함수
def convert_reposts(repost_str):
    if '만' in repost_str:
        return int(float(repost_str.replace('만', '')) * 10000)
    elif '천' in repost_str:
        return int(float(repost_str.replace('천', '')) * 1000)
    return int(repost_str)

# 답글 제한 여부 변환 함수
def convert_reply_limit(reply_limit):
    return 0 if reply_limit == '제한 없음' else 1

# 변환 적용
df1['리포스트 수'] = df1['리포스트 수'].apply(convert_reposts)
df1['답글 제한 여부'] = df1['답글 제한 여부'].apply(convert_reply_limit)

#새로운 열 '라벨링' 추가 및 값 1로 설정 << 여기 변경해주세요!! 불법이면 1, 일반이면 0
df1['라벨링'] = 0
# 리스트를 문자열로 변환
df1['전처리된 트윗_문자열'] = df1['전처리된 트윗'].apply(lambda x: ' '.join(x))

df1

,계정 이름,트윗 내용,리포스트 수,답글 제한 여부,작성한 포스트 수,전처리된 트윗,라벨링,전처리된 트윗_문자열
0,눕눕,아니.. 이거 작년 일본여행 갔을 때 사고 화장대 깊숙히 보관해뒀다가\n\n하이라이...,7700,0,"6,130","[아니다, 작년, 일본여행, 가다, 사고, 화장, 깊숙하다, 보관, 두다, 하이, ...",0,아니다 작년 일본여행 가다 사고 화장 깊숙하다 보관 두다 하이 라이터 발색 정리 려...
1,눕눕,아니 ㅁㅊ 삐아 도대체 뭘 만든거에요\n\n아 누디보틀더살걸누디보틀더살걸누디보틀더살걸,4000,0,"6,130","[아니다, 도대체, 만들다, 누디, 보틀, 더살걸누디, 보틀, 더살걸누디, 보틀, ...",0,아니다 도대체 만들다 누디 보틀 더살걸누디 보틀 더살걸누디 보틀 더살걸
2,눕눕,"이것도 진짜 예뿌죠,,\n\n색감 완전 취향저격이라 한달내내 얘만 발랐던 기억이.....",3000,0,"6,130","[진짜, 색감, 완전, 취향, 저격, 이라, 내내, 바르다, 기억, 쌩얼, 얇다, ...",0,진짜 색감 완전 취향 저격 이라 내내 바르다 기억 쌩얼 얇다 올려주다 진짜 자연스럽...
3,눕눕,요즘 저의 최애 조합입니다..\n\n클리오 양떼구름베이지 (제품제공)\n네이밍 시럽...,2000,0,"6,130","[요즘, 최애, 조합, 클리오, 구름, 베이지, 제품, 제공, 시럽, 멜팅, 스틱,...",0,요즘 최애 조합 클리오 구름 베이지 제품 제공 시럽 멜팅 스틱 리즈 제품 제공
4,눕눕,화이트~샴페인 베이지 하이라이터 모음\n\n이 중에 하나 정도는 맘에 드는게 있지 ...,2300,0,"6,130","[화이트, 샴페인, 베이지, 하이, 라이터, 모음, 하나, 정도, 들다, 않다, 발...",0,화이트 샴페인 베이지 하이 라이터 모음 하나 정도 들다 않다 발색 타래
...,...,...,...,...,...,...,...,...
195,박코덕,더샘 컨실러 팔레트 새로 나왔나봐요\n올영 단독인거같고 \n납작 브러쉬랑 스펀지 팁...,455,0,1.6만,"[더샘, 컨실러, 팔레트, 새롭다, 나오다, 보다, 단독, 인거, 같다, 납작, 브...",0,더샘 컨실러 팔레트 새롭다 나오다 보다 단독 인거 같다 납작 브러쉬 스펀지 내장 되어다
196,박코덕,릴리바이레드 탱글젤리밤 발색비교\n01 살구젤리한입(#제품제공)\n\n- AOU 땅...,858,0,1.6만,"[릴리, 바이, 레드, 젤리, 발색, 비교, 살구, 젤리, 제품, 제공, 땅콩, 네...",0,릴리 바이 레드 젤리 발색 비교 살구 젤리 제품 제공 땅콩 네이처리퍼블릭 살구 가득...
197,박코덕,근데 저거 남자 주작같긴한데\n진짜 실제 여성분이었다면 고능소비 알려주고 싶었을듯....,401,0,1.6만,"[근데, 남자, 주작, 같다, 진짜, 실제, 여성, 소비, 알다, 싶다, 너무, 아...",0,근데 남자 주작 같다 진짜 실제 여성 소비 알다 싶다 너무 아깝다 영식 정가 잖다 ...
198,박코덕,어제는 이걸로 세미매트 쿠션 두드리고\n오늘은 세미글로우 쿠션 두드렸는데요 어느제형...,657,0,1.6만,"[어제, 세미, 매트, 쿠션, 두드리다, 오늘, 세미, 로우, 쿠션, 두드리다, 어...",0,어제 세미 매트 쿠션 두드리다 오늘 세미 로우 쿠션 두드리다 어느 하고 써다 만족 ...


In [ ]:
#해시태그 살리는 코드
import pandas as pd
import re
!pip install konlpy
from konlpy.tag import Okt
from nltk.corpus import stopwords

# 파일 경로 설정
file_path = r'/content/drive/MyDrive/combined_reviews2 - 전처리 전.csv'

# CSV 파일 읽기
df2 = pd.read_csv(file_path)

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 불용어 리스트 (필요에 따라 수정)
stop_words = stoplist=['한','에서','하다','를','한다는','이라는','까지','에서는','으로','인데','이고','에요','이를','있다','여서','이다',
                       '부터','지인','짜다','라고','되었다','고함','고해']
def preprocess_tweet(tweet):
    # 링크 제거
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)

    # 해시태그 추출
    hashtags = re.findall(r'#\S+', tweet)  # 해시태그 추출
    tweet = re.sub(r'#\S+', '', tweet)  # 해시태그를 텍스트에서 제거

    # 줄바꿈 문자 및 특수 문자, 숫자 제거 (한글과 공백만 남김)
    tweet = re.sub(r'[\n\t]+|[^가-힣\s]', '', tweet)

    # 토큰화
    tokens = okt.morphs(tweet, stem=True)  # 동사 원형으로 변환

    # 불용어 제거 및 길이가 2 이상인 토큰만 필터링
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 1]

    # 해시태그를 다시 추가
    return filtered_tokens + hashtags  # 필터링된 토큰과 해시태그를 합침

# 비문자열 값을 문자열로 변환하고 NaN 값을 빈 문자열로 대체#
df2['트윗 내용'] = df2['트윗 내용'].astype(str).fillna('')

# 전처리 함수 적용
df2['전처리된 트윗'] = df2['트윗 내용'].apply(preprocess_tweet)



# 리포스트 수 변환 함수
def convert_reposts(repost_str):
    # 값이 NaN이거나 float 타입일 경우 문자열로 변환
    if pd.isna(repost_str):
        return 0
    repost_str = str(repost_str)

    if '만' in repost_str:
        return int(float(repost_str.replace('만', '')) * 10000)
    elif '천' in repost_str:
        return int(float(repost_str.replace('천', '')) * 1000)
    else:
        return int(repost_str)

# 변환 적용
df2['리포스트 수'] = df2['리포스트 수'].apply(convert_reposts)

# 답글 제한 여부 변환 함수
def convert_reply_limit(reply_limit):
    return 0 if reply_limit == '제한 없음' else 1

# 변환 적용

df2['답글 제한 여부'] = df2['답글 제한 여부'].apply(convert_reply_limit)

#새로운 열 '라벨링' 추가 및 값 1로 설정
df2['라벨링'] = 1
# 리스트를 문자열로 변환
df2['전처리된 트윗_문자열'] = df2['전처리된 트윗'].apply(lambda x: ' '.join(x))

df2

df2.to_csv('불법바이럴 리뷰.csv', index=False, encoding='utf-8-sig')
print("데이터가 파일로 저장되었습니다.")

데이터가 파일로 저장되었습니다.


In [ ]:
# df (불법 바이럴), df1 (일반 리뷰), df2 (라벨링 안 된 리뷰) 합치기
combined_df = pd.concat([df, df1, df2], ignore_index=True)

# 결과 확인
print(combined_df)

     계정 이름                                              트윗 내용  리포스트 수  \
0     숙소일기  부산 빌라쥬드 아난티\n 기장군 기장읍 기장해안로 267-7\n \n 부산에 새로 ...    7100   
1     숙소일기  후쿠오카 오션하우스\n 819-0162 Fukuoka, Nishi Ward\n \n...    4700   
2     숙소일기  베트남 호이안\n \n 이곳은 카리나 인스타에 올라왔던 숙소에요!\n 무려 5성급의...    4800   
3     숙소일기  강릉 별별별장\n 강릉시 하남길 197-18\n \n 강릉 어느 한 숲 속에 있는 ...    5100   
4     숙소일기  에어비앤비에서 한국의 한옥이 전세계적으로 인정받는 아름다운 건축물로 인정되어 카테고...    6600   
...    ...                                                ...     ...   
1284     션                                 이때 화장 맘에드는데 어케햇더라…      75   
1285     션                                    요즘은 언더래쉬 그리고 다님      75   
1286     션  투크 언더브라운!!! 저 이거 백날천날 말해요\n다른 컬러들도 아름다워섬 왕추천.\...     268   
1287     션                                     받을사람 없는 볼하트 대령      52   
1288     션                                             나를바라바~      43   

      답글 제한 여부 작성한 포스트 수                                            전처리된 트윗  \
0            0       389  [부산, 빌라, 쥬드, 아난, 기

In [ ]:
combined_df.to_csv('combined_reviews2.csv', index=False, encoding='utf-8-sig')
print("데이터가 'combined_reviews2.csv' 파일로 저장되었습니다.")

데이터가 'combined_reviews2.csv' 파일로 저장되었습니다.
